<a href="https://colab.research.google.com/github/Hmiru/Fashion-sketch-using-AI/blob/main/pose_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 사전 준비
print ("Installing Libraries...")
import os; os.system('pip install --upgrade --no-cache-dir gdown tensorfn')
os.system('pip install --upgrade --no-cache-dir pydantic==1.10.14')
print ("---> Successfully Installed.")
print ("Downloading PIDM models...")
os.system('git clone https://github.com/ankanbhunia/PIDM')
os.system('gdown 1WkV5Pn-_fBdiZlvVHHx_S97YESBkx4lD && mkdir PIDM/checkpoints && mv last.pt PIDM/checkpoints/last.pt')
print ("---> Successfully Downloaded.")
import os

!pip install gdown

import gdown

file_id = '1waNzq-deGBKATXMU9JzMDWdGsF4YkcW_'
destination_dir = '/content/PIDM/data/deepfashion_256x256/pose/'
os.makedirs(destination_dir, exist_ok=True)
destination_file = os.path.join(destination_dir, 'pose.rar')
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination_file, quiet=False)
!apt-get install unrar
!unrar x {destination_file} /content/PIDM/data/deepfashion_256x256/

Installing Libraries...
---> Successfully Installed.
---> Successfully Downloaded.


In [ ]:
# @title image error가 뜨면 클릭
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms.functional as F
import cv2
import math
import matplotlib.pyplot as plt
from io import BytesIO
#from PIDM.predict import *
import sys
import sys
sys.path.append('/Untitled Forder/PIDM/config')

# get_label_tensor 함수 정의
def get_label_tensor(path, shape=[256, 256]):
    # 텍스트 파일에서 키포인트 데이터 로드
    keypoint = np.loadtxt(path)
    keypoint = trans_keypoins(keypoint, shape)

    # 캔버스 생성 및 키포인트 그리기
    canvas = np.zeros((shape[0], shape[1], 3)).astype(np.uint8)
    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0],
              [85, 255, 0], [0, 255, 0], [0, 255, 85], [0, 255, 170], [0, 255, 255],
              [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], [170, 0, 255],
              [255, 0, 255], [255, 0, 170], [255, 0, 85]]
    limbSeq = [[2, 3], [2, 6], [3, 4], [4, 5], [6, 7], [7, 8], [2, 9], [9, 10],
               [10, 11], [2, 12], [12, 13], [13, 14], [2, 1], [1, 15], [15, 17],
               [1, 16], [16, 18], [3, 17], [6, 18]]
    stickwidth = 4

    for i in range(18):
        x, y = keypoint[i, 0:2]
        if x == -1 or y == -1:
            continue
        cv2.circle(canvas, (int(x), int(y)), 4, colors[i], thickness=-1)

    joints = []
    for i in range(17):
        Y = keypoint[np.array(limbSeq[i])-1, 0]
        X = keypoint[np.array(limbSeq[i])-1, 1]
        cur_canvas = canvas.copy()
        if -1 in Y or -1 in X:
            joints.append(np.zeros_like(cur_canvas[:, :, 0]))
            continue
        mX = np.mean(X)
        mY = np.mean(Y)
        length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
        angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
        polygon = cv2.ellipse2Poly((int(mY), int(mX)), (int(length / 2), stickwidth), int(angle), 0, 360, 1)
        cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
        canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)

        joint = np.zeros_like(cur_canvas[:, :, 0])
        cv2.fillConvexPoly(joint, polygon, 255)
        joint = cv2.addWeighted(joint, 0.4, joint, 0.6, 0)
        joints.append(joint)

    pose = F.to_tensor(Image.fromarray(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB)))
    tensors_dist = 0
    e = 1

    for i in range(len(joints)):
        im_dist = cv2.distanceTransform(255 - joints[i], cv2.DIST_L1, 3)
        im_dist = np.clip((im_dist / 3), 0, 255).astype(np.uint8)
        tensor_dist = F.to_tensor(Image.fromarray(im_dist))
        tensors_dist = tensor_dist if e == 1 else torch.cat([tensors_dist, tensor_dist])
        e += 1

    label_tensor = torch.cat((pose, tensors_dist), dim=0)

    if int(keypoint[14, 0]) != -1 and int(keypoint[15, 0]) != -1:
        y0, x0 = keypoint[14, 0:2]
        y1, x1 = keypoint[15, 0:2]
        face_center = torch.tensor([y0, x0, y1, x1]).float()
    else:
        face_center = torch.tensor([-1, -1, -1, -1]).float()

    return label_tensor, face_center

# 키포인트 변환 함수 정의
def trans_keypoins(keypoints, img_size):
    missing_keypoint_index = keypoints == -1

    # crop the white line in the original dataset
    keypoints[:,0] = (keypoints[:,0] - 40)

    # resize the dataset
    img_w, img_h = img_size
    scale_h = 1.0 / 176.0 * img_h
    scale_w = 1.0 / 256.0 * img_w

    keypoints[:,0] = keypoints[:,0] * scale_h
    keypoints[:,1] = keypoints[:,1] * scale_w
    keypoints[missing_keypoint_index] = -1
    return keypoints

def load_image_as_url(path):
    img = Image.open(path)
    buf = BytesIO()
    img.save(buf, format="PNG")
    byte_im = buf.getvalue()
    return byte_im

def adjust_tensor_shape(label_tensor):
    # Permute the tensor from (20, 256, 256) to (256, 256, 20)
    adjusted_tensor = label_tensor.permute(1, 2, 0).numpy()
    return adjusted_tensor

주어진 사진의 의상에 따라 포즈 스켈레톤이 살짝씩 다릅니다.

In [ ]:
# @title 기본 실행

import shutil
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

def list_directories(base_path):
    return [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

def list_files(base_path, clothing_type, clothing_subtype, clothing_id):
    target_path = os.path.join(base_path, clothing_type, clothing_subtype, clothing_id)

    if not os.path.exists(target_path):
        print(f"The specified path does not exist: {target_path}")
        return None

    result = []
    for root, dirs, files in os.walk(target_path):
        for file in files:
            result.append(os.path.join(root, file))

    return result

pose_path = "/content/PIDM/data/deepfashion_256x256/pose"
directory_path = "/content/PIDM/data/deepfashion_256x256/target_pose/"

# Global variable to store file structure
files_structure = []

# Widgets
clothing_type_dropdown = widgets.Dropdown(options=[], description='Clothing Type:')
clothing_subtype_dropdown = widgets.Dropdown(options=[], description='Clothing Subtype:')
clothing_id_dropdown = widgets.Dropdown(options=[], description='Clothing ID:')
output = widgets.Output()
load_button = widgets.Button(description="Load Files")
convert_button = widgets.Button(description="Convert and Save")

def update_clothing_subtypes(change):
    if change['new'] is not None:
        clothing_subtypes = list_directories(os.path.join(pose_path, change['new']))
        clothing_subtype_dropdown.options = clothing_subtypes
        clothing_subtype_dropdown.value = None
        clothing_id_dropdown.options = []
        with output:
            clear_output()
            print("Please select a clothing subtype.")

def update_clothing_ids(change):
    if change['new'] is not None:
        clothing_ids = list_directories(os.path.join(pose_path, clothing_type_dropdown.value, change['new']))
        clothing_id_dropdown.options = clothing_ids
        clothing_id_dropdown.value = None
        with output:
            clear_output()
            print("Please select a clothing ID.")

def load_files(b):
    global files_structure
    if clothing_id_dropdown.value is not None:
        files_structure = list_files(pose_path, clothing_type_dropdown.value, clothing_subtype_dropdown.value, clothing_id_dropdown.value)
        with output:
            clear_output()
            if files_structure:
                for file_path in files_structure:
                    print(file_path)
            else:
                print("No files found or invalid path.")
    else:
        with output:
            clear_output()
            print("Please select a clothing ID.")

def convert_and_save_files(b):
    global files_structure
    # 폴더가 존재하는지 확인
    if os.path.exists(directory_path):
        # 폴더 안의 모든 파일 및 하위 폴더 삭제
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
        print(f"All contents of the directory '{directory_path}' have been deleted.")
    else:
        os.makedirs(directory_path)
        print(f"The directory '{directory_path}' has been created.")

    if files_structure:
        text_files = [f for f in files_structure if f.endswith('txt')]

        for txt in text_files:
            label_tensor, _ = get_label_tensor(txt)  # get_label_tensor 함수가 정의되어 있어야 합니다.
            adjusted_tensor = adjust_tensor_shape(label_tensor)  # adjust_tensor_shape 함수가 정의되어 있어야 합니다.
            save_path = os.path.join(directory_path, os.path.basename(txt).replace('.txt', '.npy'))
            np.save(save_path, adjusted_tensor)

        print(f"모든 텍스트 파일이 {directory_path} 경로에 npy 파일로 변환되어 저장되었습니다.")
    else:
        print("No files found or invalid path.")

# Initialize clothing types
clothing_types = list_directories(pose_path)
clothing_type_dropdown.options = clothing_types

# Event listeners
clothing_type_dropdown.observe(update_clothing_subtypes, names='value')
clothing_subtype_dropdown.observe(update_clothing_ids, names='value')
load_button.on_click(load_files)
convert_button.on_click(convert_and_save_files)

# Display widgets
display(clothing_type_dropdown, clothing_subtype_dropdown, clothing_id_dropdown, load_button, convert_button, output)


Dropdown(description='Clothing Type:', options=('WOMEN', 'MEN'), value='WOMEN')

Dropdown(description='Clothing Subtype:', options=(), value=None)

Dropdown(description='Clothing ID:', options=(), value=None)

Button(description='Load Files', style=ButtonStyle())

Button(description='Convert and Save', style=ButtonStyle())

Output()

All contents of the directory '/content/PIDM/data/deepfashion_256x256/target_pose/' have been deleted.
모든 텍스트 파일이 /content/PIDM/data/deepfashion_256x256/target_pose/ 경로에 npy 파일로 변환되어 저장되었습니다.


In [ ]:
# @title ram이 부족할 때 클릭
import gc
gc.collect()


0

In [ ]:
#@title 원하는 사진을 넣어서 각도 조정 확인하기
%cd "/content/PIDM"
from predict import Predictor
from IPython.display import Image
from google.colab import files
obj = Predictor()
npy_files = [f for f in os.listdir(directory_path) if f.endswith('.npy')]
npy_count = len(npy_files)

# ### upload a source file
Path = list(files.upload().keys())[0] # Please check "/content/PIDM/data/" for sample images for source file.
# ### OR enter the path
#Path = "data/deepfashion_256x256/target_edits/reference_img_3.png"

obj.predict_pose(image=Path, sample_algorithm='ddim', num_poses=npy_count, nsteps=50)

Image('output.png')